In [ ]:
! pip install  transformers faiss-cpu sentence-transformers

In [ ]:
import json
import os

# List of file names you want to combine
file_names = ['MVA.json', 'cpc.json', 'crpc_new.json','ida.json','iea_new.json','ipc_new.json','nia_new.json']

# Initialize an empty list to hold all the data
combined_data = []

# Loop over the list of files and read in the data
for file_name in file_names:
    with open(file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)
        # Remove the 'section' key from each entry
        for entry in data:
            if 'section' in entry:
                del entry['section']
        # Assuming each file contains a list of entries
        combined_data.extend(data)
# Encode the data for RAG use
with open('combined.json', 'w', encoding='utf-8') as file:
    json.dump(combined_data, file, ensure_ascii=False, indent=4)

In [ ]:
import json

# Assuming 'combined.json' is the file with combined entries without 'section' key
combined_file_name = 'combined.json'
jsonl_file_name = 'combined.jsonl'

# Read the combined JSON file
with open(combined_file_name, 'r', encoding='utf-8') as json_file:
    combined_data = json.load(json_file)

# Write each entry as a single line in the JSONL file
with open(jsonl_file_name, 'w', encoding='utf-8') as jsonl_file:
    for entry in combined_data:
        jsonl_file.write(json.dumps(entry, ensure_ascii=False) + '\n')

In [ ]:
from sentence_transformers import SentenceTransformer, util
import json
import numpy as np
import faiss

# Specify the model you wish to use
model_name = 'sentence-transformers/all-mpnet-base-v2'  # This model requires less memory and is faster

# Load the .jsonl file and extract the text
def load_jsonl(file_path):
    texts = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            entry = json.loads(line)
            text = entry.get("description", "").strip()
            if text:
                texts.append(text)
    return texts

# Generate embeddings for a list of texts
def get_embeddings(texts):
    model = SentenceTransformer(model_name)
    return model.encode(texts, show_progress_bar=True)

# Load texts from the .jsonl file
jsonl_file_path = 'combined.jsonl'
texts = load_jsonl(jsonl_file_path)

# Generate embeddings
embeddings = get_embeddings(texts)

# FAISS index creation
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings).astype('float32'))

# Function to retrieve documents using FAISS index
def retrieve_documents(query, k=5):
    query_embedding = get_embeddings([query])[0]
    distances, indices = index.search(np.array([query_embedding]).astype('float32'), k)
    return [texts[i] for i in indices[0]]

# # Example usage

# print(retrieved_docs)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
llm_model_name = 'allenai/unifiedqa-v2-t5-large-1363200'  # Example model, you can use any model you prefer
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(llm_model_name)

# Generate a response with the LLM

# Generate a response with the LLM with adjusted parameters
def generate_response(input_text):
    inputs = tokenizer.encode("context: " + input_text, return_tensors="pt", add_special_tokens=True)

    # Adjust generation parameters
    outputs = llm_model.generate(
        inputs,
        max_length=300,  # Increase max_length for potentially longer output
        min_length=150,   # Set a minimum length to ensure a longer response
        temperature=0.9,  # Adjust temperature if you want more creative responses
        num_beams=10,     # Increase the number of beams for more thorough search
        no_repeat_ngram_size=2,  # Prevent repeating the same information
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Retrieve documents related to the query

query = 'What is fraud?'
retrieved_docs = retrieve_documents(query,k=3)

# Construct the prompt
prompt = "You have detailed knowledge of Indian Law. Here are a list of related documents:" + "\n".join(retrieved_docs) + "\nPlease write the most relevant answer to the next question" + query

# Generate the response using the LLM
response = generate_response(prompt)

print(response)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


fraud is when someone intentionally does or omits to do something which would not happen if he were not so deceived, and which cause or is likely to cause damage or harm to that person in body, mind, reputation or property, is said to “cheat”. Please write the most relevant answer to the next questionWhat is fraud? ie, when a person intentionally performs an act which results in damage, harm, or loss of property.said to "chean when the person does, s n re, in -- &non-asa l  ens,re-lon,aa
